In [12]:
# Importazione delle librerie necessarie per il grafo di conoscenza
import pandas as pd
import json
import os
from pathlib import Path
from neo4j import GraphDatabase
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("Librerie importate con successo per la creazione del grafo di conoscenza")

Librerie importate con successo per la creazione del grafo di conoscenza


In [13]:
# Configurazione della connessione a Neo4j
# Avviarlo con Docker: docker run -p 7474:7474 -p 7687:7687 neo4j:latest

NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"  # Cambia con la tua password

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        
    def close(self):
        self.driver.close()
        
    def clear_database(self):
        """Pulisce il database per ricominciare da capo"""
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")
            logger.info("Database pulito")
    
    def create_indexes(self):
        """Crea gli indici per migliorare le performance"""
        with self.driver.session() as session:
            # Indici per i prodotti
            session.run("CREATE INDEX product_id IF NOT EXISTS FOR (p:Product) ON (p.product_id)")
            session.run("CREATE INDEX product_name IF NOT EXISTS FOR (p:Product) ON (p.name)")
            
            # Indici per i documenti
            session.run("CREATE INDEX document_filename IF NOT EXISTS FOR (d:Document) ON (d.filename)")
            
            # Indici per le annotazioni
            session.run("CREATE INDEX annotation_filename IF NOT EXISTS FOR (a:Annotation) ON (a.filename)")
            
            logger.info("Indici creati")

print("Classe KnowledgeGraphBuilder definita")


Classe KnowledgeGraphBuilder definita


In [14]:
# Caricamento e preparazione dei dati CSV
def load_csv_data():
    """Carica tutti i file CSV da AdventureWorks"""
    data_path = Path("../data")
    
    # Leggiamo i CSV principali
    products_df = pd.read_csv(data_path / "Product.csv", sep=';')
    categories_df = pd.read_csv(data_path / "ProductCategory.csv", sep=';')
    descriptions_df = pd.read_csv(data_path / "ProductDescription.csv", sep=';')
    models_df = pd.read_csv(data_path / "ProductModel.csv", sep=';')
    
    logger.info(f"Caricati {len(products_df)} prodotti")
    logger.info(f"Caricate {len(categories_df)} categorie")
    logger.info(f"Caricate {len(descriptions_df)} descrizioni")
    logger.info(f"Caricati {len(models_df)} modelli")
    
    return {
        'products': products_df,
        'categories': categories_df,
        'descriptions': descriptions_df,
        'models': models_df
    }

# Carica i dati
csv_data = load_csv_data()

# Esplora i primi record per vedere la struttura
print("\\nPrimi 3 prodotti:")
print(csv_data['products'][['ProductID', 'Name', 'ProductCategoryID', 'ProductModelID']].head(3))

print("\\nPrime 5 categorie:")
print(csv_data['categories'][['ProductCategoryID', 'ParentProductCategoryID', 'Name']].head())


INFO:__main__:Caricati 100 prodotti
INFO:__main__:Caricate 41 categorie
INFO:__main__:Caricate 100 descrizioni
INFO:__main__:Caricati 100 modelli


\nPrimi 3 prodotti:
   ProductID                       Name  ProductCategoryID  ProductModelID
0        680  HL Road Frame - Black, 58                 18               6
1        706    HL Road Frame - Red, 58                 18               6
2        707      Sport-100 Helmet, Red                 35              33
\nPrime 5 categorie:
   ProductCategoryID  ParentProductCategoryID            Name
0                  1                      NaN           Bikes
1                  2                      NaN      Components
2                  3                      NaN        Clothing
3                  4                      NaN     Accessories
4                  5                      1.0  Mountain Bikes


In [15]:
# Analisi dei documenti e annotazioni in IngestedDocuments
def analyze_ingested_documents():
    """Analizza la struttura dei documenti e delle loro annotazioni"""
    docs_path = Path("../data/IngestedDocuments")
    
    # Raggruppa i file per base name (senza estensione e senza suffissi)
    files = list(docs_path.glob("*"))
    documents = {}
    
    for file in files:
        # Estrae il nome del documento
        name = file.name
        
        # Identifica il tipo di file
        if name.endswith('.pdf'):
            base_name = name.replace('.pdf', '')
            if base_name not in documents:
                documents[base_name] = {'pdf': None, 'annotations': []}
            documents[base_name]['pdf'] = file
            
        elif name.endswith('.jpg'):
            # Rimuove " Fig X" dalla fine se presente
            base_name = name.replace('.jpg', '')
            if ' Fig ' in base_name:
                base_name = base_name.split(' Fig ')[0]
            if base_name not in documents:
                documents[base_name] = {'pdf': None, 'annotations': []}
            documents[base_name]['annotations'].append(file)
            
        elif name.endswith('.json'):
            # Rimuove " Table X" dalla fine se presente
            base_name = name.replace('.json', '')
            if ' Table ' in base_name:
                base_name = base_name.split(' Table ')[0]
            if base_name not in documents:
                documents[base_name] = {'pdf': None, 'annotations': []}
            documents[base_name]['annotations'].append(file)
    
    logger.info(f"Trovati {len(documents)} gruppi di documenti")
    
    for doc_name, doc_data in documents.items():
        print(f"\\nDocumento: {doc_name}")
        print(f"  PDF: {'Sì' if doc_data['pdf'] else 'Mancante'}")
        print(f"  Annotazioni: {len(doc_data['annotations'])}")
        for ann in doc_data['annotations']:
            print(f"    - {ann.name}")
    
    return documents

# Analizza i documenti
document_structure = analyze_ingested_documents()


INFO:__main__:Trovati 4 gruppi di documenti


\nDocumento: Vintage Trailblazer X-1 Mountain Bike (1995)
  PDF: Sì
  Annotazioni: 3
    - Vintage Trailblazer X-1 Mountain Bike (1995) Table 1.json
    - Vintage Trailblazer X-1 Mountain Bike (1995).jpg
    - Vintage Trailblazer X-1 Mountain Bike (1995) Table 2.json
\nDocumento: LL Mountain Handlebars (Black)
  PDF: Sì
  Annotazioni: 2
    - LL Mountain Handlebars (Black) Table 1.json
    - LL Mountain Handlebars (Black) Fig 1.jpg
\nDocumento: Long-Sleeve Logo Jersey (M)
  PDF: Sì
  Annotazioni: 2
    - Long-Sleeve Logo Jersey (M) Table 1.json
    - Long-Sleeve Logo Jersey (M) Fig 1.jpg
\nDocumento: Mountain Bike Manual
  PDF: Sì
  Annotazioni: 2
    - Mountain Bike Manual Table 1.json
    - Mountain Bike Manual Table 2.json


In [16]:
# Estensione della classe KnowledgeGraphBuilder con metodi per creare i nodi
class KnowledgeGraphBuilder(KnowledgeGraphBuilder):
    
    def create_product_nodes(self, products_df, categories_df, models_df):
        """Crea i nodi per i prodotti con le loro proprietà"""
        with self.driver.session() as session:
            for _, product in products_df.iterrows():
                try:
                    # Trova la categoria (gestisce i valori NaN)
                    category_name = 'Unknown'
                    if pd.notna(product.get('ProductCategoryID')):
                        category = categories_df[categories_df['ProductCategoryID'] == product['ProductCategoryID']]
                        if not category.empty:
                            category_name = str(category['Name'].iloc[0])
                    
                    # Trova il modello (gestisce i valori NaN)
                    model_name = 'Unknown'
                    if pd.notna(product.get('ProductModelID')):
                        model = models_df[models_df['ProductModelID'] == product['ProductModelID']]
                        if not model.empty:
                            model_name = str(model['Name'].iloc[0])
                    
                    # Crea il nodo prodotto
                    query = '''
                    CREATE (p:Product {
                        product_id: $product_id,
                        name: $name,
                        product_number: $product_number,
                        color: $color,
                        standard_cost: $standard_cost,
                        list_price: $list_price,
                        size: $size,
                        weight: $weight,
                        category_id: $category_id,
                        category_name: $category_name,
                        model_id: $model_id,
                        model_name: $model_name,
                        sell_start_date: $sell_start_date
                    })
                    '''
                    
                    # Prepara i parametri con gestione sicura dei tipi
                    params = {
                        'product_id': int(product['ProductID']),
                        'name': str(product['Name']),
                        'product_number': str(product['ProductNumber']),
                        'color': str(product.get('Color', '') if pd.notna(product.get('Color')) else ''),
                        'standard_cost': float(str(product.get('StandardCost', '0')).replace(',', '.')) if pd.notna(product.get('StandardCost')) else 0.0,
                        'list_price': float(str(product.get('ListPrice', '0')).replace(',', '.')) if pd.notna(product.get('ListPrice')) else 0.0,
                        'size': str(product.get('Size', '') if pd.notna(product.get('Size')) else ''),
                        'weight': str(product.get('Weight', '') if pd.notna(product.get('Weight')) else ''),
                        'category_id': int(product['ProductCategoryID']) if pd.notna(product.get('ProductCategoryID')) else None,
                        'category_name': category_name,
                        'model_id': int(product['ProductModelID']) if pd.notna(product.get('ProductModelID')) else None,
                        'model_name': model_name,
                        'sell_start_date': str(product.get('SellStartDate', '') if pd.notna(product.get('SellStartDate')) else '')
                    }
                    
                    session.run(query, **params)
                    
                except Exception as e:
                    logger.error(f"Errore nel creare il nodo per il prodotto {product.get('ProductID', 'Unknown')}: {e}")
                    continue
            
            logger.info(f"Creati {len(products_df)} nodi Product")

print("Metodo create_product_nodes aggiunto alla classe")


Metodo create_product_nodes aggiunto alla classe


In [17]:
# Aggiunta di metodi per creare documenti e annotazioni
class KnowledgeGraphBuilder(KnowledgeGraphBuilder):
    
    def create_document_nodes(self, document_structure):
        """Crea i nodi per i documenti PDF e le loro annotazioni"""
        with self.driver.session() as session:
            for doc_name, doc_data in document_structure.items():
                
                # Crea il nodo documento PDF se esiste
                if doc_data['pdf']:
                    pdf_file = doc_data['pdf']
                    
                    query = '''
                    CREATE (d:Document {
                        filename: $filename,
                        document_name: $document_name,
                        file_path: $file_path,
                        file_type: 'PDF',
                        file_size: $file_size
                    })
                    '''
                    
                    session.run(query,
                               filename=pdf_file.name,
                               document_name=doc_name,
                               file_path=str(pdf_file),
                               file_size=pdf_file.stat().st_size if pdf_file.exists() else 0
                    )
                    
                    # Crea i nodi annotazione e le relazioni
                    for annotation_file in doc_data['annotations']:
                        ann_type = 'Image' if annotation_file.suffix == '.jpg' else 'Table'
                        
                        # Leggi il contenuto se è un JSON
                        content = None
                        if annotation_file.suffix == '.json':
                            try:
                                with open(annotation_file, 'r') as f:
                                    content = json.load(f)
                            except Exception as e:
                                logger.warning(f"Errore nel leggere {annotation_file}: {e}")
                        
                        # Crea il nodo annotazione
                        ann_query = '''
                        CREATE (a:Annotation {
                            filename: $filename,
                            annotation_type: $annotation_type,
                            file_path: $file_path,
                            content: $content,
                            file_size: $file_size
                        })
                        '''
                        
                        session.run(ann_query,
                                   filename=annotation_file.name,
                                   annotation_type=ann_type,
                                   file_path=str(annotation_file),
                                   content=json.dumps(content) if content else None,
                                   file_size=annotation_file.stat().st_size if annotation_file.exists() else 0
                        )
                        
                        # Crea la relazione ANNOTATION tra documento e annotazione
                        rel_query = '''
                        MATCH (d:Document {filename: $doc_filename})
                        MATCH (a:Annotation {filename: $ann_filename})
                        CREATE (a)-[:ANNOTATION]->(d)
                        '''
                        
                        session.run(rel_query,
                                   doc_filename=pdf_file.name,
                                   ann_filename=annotation_file.name
                        )
            
            logger.info(f"Creati nodi per {len(document_structure)} documenti e relative annotazioni")

print("Metodo create_document_nodes aggiunto alla classe")


Metodo create_document_nodes aggiunto alla classe


In [18]:
# Aggiunta di metodi per creare relazioni tra prodotti
class KnowledgeGraphBuilder(KnowledgeGraphBuilder):
    
    def create_product_relationships(self):
        """Crea relazioni statiche/manuali tra prodotti"""
        with self.driver.session() as session:
            
            # 1. Relazioni SAME_CATEGORY tra prodotti della stessa categoria
            category_query = '''
            MATCH (p1:Product), (p2:Product)
            WHERE p1.category_id = p2.category_id 
            AND p1.product_id <> p2.product_id
            AND p1.category_id IS NOT NULL
            CREATE (p1)-[:SAME_CATEGORY]->(p2)
            '''
            result1 = session.run(category_query)
            logger.info("Relazioni SAME_CATEGORY create")
            
            # 2. Relazioni SAME_MODEL tra prodotti dello stesso modello
            model_query = '''
            MATCH (p1:Product), (p2:Product)
            WHERE p1.model_id = p2.model_id 
            AND p1.product_id <> p2.product_id
            AND p1.model_id IS NOT NULL
            CREATE (p1)-[:SAME_MODEL]->(p2)
            '''
            result2 = session.run(model_query)
            logger.info("Relazioni SAME_MODEL create")
            
            # 3. Relazioni SIMILAR_PRICE tra prodotti con prezzi simili (±20%)
            price_query = '''
            MATCH (p1:Product), (p2:Product)
            WHERE p1.product_id <> p2.product_id
            AND p1.list_price > 0 AND p2.list_price > 0
            AND abs(p1.list_price - p2.list_price) / p1.list_price <= 0.20
            CREATE (p1)-[:SIMILAR_PRICE]->(p2)
            '''
            result3 = session.run(price_query)
            logger.info("Relazioni SIMILAR_PRICE create")
            
            # 4. Relazioni manuali specifiche per prodotti molto correlati
            # Esempio: tutti i frame road sono correlati
            manual_relations = [
                {
                    'filter1': "p1.name CONTAINS 'Road Frame'",
                    'filter2': "p2.name CONTAINS 'Road Frame'",
                    'relation': 'COMPATIBLE_PRODUCT'
                },
                {
                    'filter1': "p1.name CONTAINS 'Mountain'",
                    'filter2': "p2.name CONTAINS 'Mountain'",
                    'relation': 'COMPATIBLE_PRODUCT'
                },
                {
                    'filter1': "p1.name CONTAINS 'Helmet'",
                    'filter2': "p2.name CONTAINS 'Jersey'",
                    'relation': 'COMPLEMENTARY_PRODUCT'
                },
                {
                    'filter1': "p1.name CONTAINS 'Frame'",
                    'filter2': "p2.name CONTAINS 'Handlebars'",
                    'relation': 'COMPLEMENTARY_PRODUCT'
                }
            ]
            
            for relation in manual_relations:
                manual_query = f'''
                MATCH (p1:Product), (p2:Product)
                WHERE {relation['filter1']}
                AND {relation['filter2']}
                AND p1.product_id <> p2.product_id
                CREATE (p1)-[:{relation['relation']}]->(p2)
                '''
                session.run(manual_query)
                logger.info(f"Relazioni {relation['relation']} create")

print("Metodo create_product_relationships aggiunto alla classe")


Metodo create_product_relationships aggiunto alla classe


In [19]:
# Metodi per collegare prodotti con documenti e query del grafo
class KnowledgeGraphBuilder(KnowledgeGraphBuilder):
    
    def create_product_document_relationships(self):
        """Crea relazioni tra prodotti e documenti basate sui nomi"""
        with self.driver.session() as session:
            
            # Cerca di collegare prodotti con documenti basandosi sui nomi
            # Ad esempio, "LL Mountain Handlebars" dovrebbe collegarsi con il documento PDF
            
            connect_query = '''
            MATCH (p:Product), (d:Document)
            WHERE d.document_name CONTAINS p.name 
            OR p.name CONTAINS d.document_name
            OR (d.document_name CONTAINS 'Mountain' AND p.name CONTAINS 'Mountain')
            OR (d.document_name CONTAINS 'Handlebars' AND p.name CONTAINS 'Handlebars')
            OR (d.document_name CONTAINS 'Jersey' AND p.name CONTAINS 'Jersey')
            CREATE (p)-[:DESCRIBED_BY]->(d)
            '''
            
            session.run(connect_query)
            logger.info("Relazioni DESCRIBED_BY tra prodotti e documenti create")
    
    def get_graph_statistics(self):
        """Restituisce statistiche sul grafo creato"""
        with self.driver.session() as session:
            stats = {}
            
            # Conta i nodi per tipo
            node_counts = session.run('''
            MATCH (n)
            RETURN labels(n)[0] as node_type, count(n) as count
            ORDER BY count DESC
            ''')
            
            stats['nodes'] = {record['node_type']: record['count'] for record in node_counts}
            
            # Conta le relazioni per tipo
            rel_counts = session.run('''
            MATCH ()-[r]->()
            RETURN type(r) as relationship_type, count(r) as count
            ORDER BY count DESC
            ''')
            
            stats['relationships'] = {record['relationship_type']: record['count'] for record in rel_counts}
            
            return stats
    
    def query_similar_products(self, product_id, limit=5):
        """Trova prodotti simili a quello specificato"""
        with self.driver.session() as session:
            query = '''
            MATCH (p:Product {product_id: $product_id})-[r]-(similar:Product)
            RETURN similar.name as product_name, 
                   similar.product_id as product_id,
                   type(r) as relationship_type,
                   similar.list_price as price
            ORDER BY similar.list_price
            LIMIT $limit
            '''
            
            result = session.run(query, product_id=product_id, limit=limit)
            return [dict(record) for record in result]

print("Metodi finali aggiunti alla classe KnowledgeGraphBuilder")


Metodi finali aggiunti alla classe KnowledgeGraphBuilder


In [22]:
# COSTRUZIONE DEL GRAFO DI CONOSCENZA

print("=== COSTRUZIONE DEL GRAFO DI CONOSCENZA ===")
print()
print("🔄 Questo script:")
print("   1. Legge i dati dai tuoi file CSV")
print("   2. Li carica in Neo4j come nodi del grafo")
print("   3. Crea relazioni intelligenti tra i nodi")
print("   4. Risultato: una knowledge base interrogabile!")
print()
print("🌐 URL Browser Neo4j: http://localhost:7474")
print("🔑 Login: neo4j / password")
print()

# Test della connessione prima di procedere
def test_neo4j_connection():
    try:
        test_driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
        with test_driver.session() as session:
            result = session.run("RETURN 'Neo4j is connected!' as message")
            message = result.single()['message']
            test_driver.close()
            return True, message
    except Exception as e:
        return False, str(e)

connected, message = test_neo4j_connection()

if connected:
    print(f"✅ {message}")
    build_graph = True  # Automaticamente procediamo se la connessione funziona
else:
    print(f"❌ Errore di connessione: {message}")
    print()
    print("🐳 Per avviare Neo4j con Docker:")
    print("docker run -p 7474:7474 -p 7687:7687 -d --env NEO4J_AUTH=neo4j/password neo4j:latest")
    print()
    print("Poi riavvia questa cella!")
    build_graph = False

if build_graph:
    try:
        # Inizializza il builder
        kg_builder = KnowledgeGraphBuilder(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
        
        print("1. Pulizia del database...")
        kg_builder.clear_database()
        
        print("2. Creazione degli indici...")
        kg_builder.create_indexes()
        
        print("3. Creazione dei nodi prodotto...")
        kg_builder.create_product_nodes(
            csv_data['products'], 
            csv_data['categories'], 
            csv_data['models']
        )
        
        print("4. Creazione dei nodi documento e annotazioni...")
        kg_builder.create_document_nodes(document_structure)
        
        print("5. Creazione delle relazioni tra prodotti...")
        kg_builder.create_product_relationships()
        
        print("6. Collegamento prodotti con documenti...")
        kg_builder.create_product_document_relationships()
        
        print("7. Statistiche del grafo creato:")
        stats = kg_builder.get_graph_statistics()
        print(f"   Nodi: {stats['nodes']}")
        print(f"   Relazioni: {stats['relationships']}")
        
        print("\\n=== GRAFO COMPLETATO! ===")
        print("Puoi ora esplorare il grafo usando Neo4j Browser su http://localhost:7474")
        print()
        print("🔍 Esempi di query Cypher per esplorare il grafo:")
        print("MATCH (n) RETURN count(n) as total_nodes")
        print("MATCH (p:Product) RETURN p.name, p.category_name LIMIT 10")
        print("MATCH (d:Document)<-[:ANNOTATION]-(a:Annotation) RETURN d.document_name, count(a) as annotations")
        
        kg_builder.close()
        
    except Exception as e:
        print(f"ERRORE: {e}")
        print("Verifica che Neo4j sia in esecuzione e che le credenziali siano corrette.")
else:
    print("⚠️ Neo4j non è connesso.")
    print("🐳 Per avviare Neo4j con Docker:")
    print("docker run -p 7474:7474 -p 7687:7687 -d --env NEO4J_AUTH=neo4j/password neo4j:latest")


INFO:__main__:Database pulito


=== COSTRUZIONE DEL GRAFO DI CONOSCENZA ===

🔄 Questo script:
   1. Legge i dati dai tuoi file CSV
   2. Li carica in Neo4j come nodi del grafo
   3. Crea relazioni intelligenti tra i nodi
   4. Risultato: una knowledge base interrogabile!

🌐 URL Browser Neo4j: http://localhost:7474
🔑 Login: neo4j / password

✅ Neo4j is connected!
1. Pulizia del database...
2. Creazione degli indici...


INFO:__main__:Indici creati
ERROR:__main__:Errore nel creare il nodo per il prodotto 680: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 706: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 712: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 713: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 714: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 715: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 716: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 717: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 718: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 719: 'Name'


3. Creazione dei nodi prodotto...


ERROR:__main__:Errore nel creare il nodo per il prodotto 720: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 721: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 722: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 723: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 724: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 725: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 726: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 727: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 728: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 729: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 730: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 731: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 732: 'Name'
ERROR:__main__:Errore nel creare il nodo per il prodotto 733: 'Name'
ERROR:__main__:Errore nel creare i

4. Creazione dei nodi documento e annotazioni...


INFO:__main__:Creati nodi per 4 documenti e relative annotazioni
INFO:__main__:Relazioni SAME_CATEGORY create


5. Creazione delle relazioni tra prodotti...


INFO:neo4j.notifications:Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Statement.CartesianProduct} {category: PERFORMANCE} {title: This query builds a cartesian product between disconnected patterns.} {description: If a part of a query contains multiple disconnected patterns, this will build a cartesian product between all those parts. This may produce a large amount of data and slow down query processing. While occasionally intended, it may often be possible to reformulate the query that avoids the use of this cross product, perhaps by adding a relationship between the different parts or by using OPTIONAL MATCH (identifier is: (p2))} {position: line: 2, column: 13, offset: 13} for query: '\n            MATCH (p1:Product), (p2:Product)\n            WHERE p1.category_id = p2.category_id \n            AND p1.product_id <> p2.product_id\n            AND p1.category_id IS NOT NULL\n            CREATE (p1)-[:SAME_CATEGORY]->(p2)\n            '

6. Collegamento prodotti con documenti...
7. Statistiche del grafo creato:
   Nodi: {'Product': 61, 'Annotation': 9, 'Document': 4}
   Relazioni: {'SAME_CATEGORY': 1510, 'SIMILAR_PRICE': 572, 'COMPATIBLE_PRODUCT': 380, 'SAME_MODEL': 354, 'DESCRIBED_BY': 60, 'ANNOTATION': 9}
\n=== GRAFO COMPLETATO! ===
Puoi ora esplorare il grafo usando Neo4j Browser su http://localhost:7474

🔍 Esempi di query Cypher per esplorare il grafo:
MATCH (n) RETURN count(n) as total_nodes
MATCH (p:Product) RETURN p.name, p.category_name LIMIT 10
MATCH (d:Document)<-[:ANNOTATION]-(a:Annotation) RETURN d.document_name, count(a) as annotations


In [10]:

#MATCH (n)-[r:DESCRIBED_BY|ANNOTATION]-(m) 
#RETURN n, r, m

In [11]:
# ESEMPI DI QUERY SUL GRAFO
# Questi esempi funzionano solo dopo aver costruito il grafo

def demo_queries():
    """Esegue alcune query di esempio sul grafo"""
    try:
        kg_builder = KnowledgeGraphBuilder(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
        
        print("=== ESEMPI DI QUERY SUL GRAFO ===\\n")
        
        # 1. Trova un prodotto e i suoi simili
        print("1. Prodotti simili al primo helmet:")
        with kg_builder.driver.session() as session:
            # Trova il primo helmet
            helmet_result = session.run("""
            MATCH (p:Product) 
            WHERE p.name CONTAINS 'Helmet' 
            RETURN p.product_id as id, p.name as name 
            LIMIT 1
            """)
            
            helmet = helmet_result.single()
            if helmet:
                print(f"   Prodotto base: {helmet['name']} (ID: {helmet['id']})")
                
                # Trova prodotti simili
                similar = kg_builder.query_similar_products(helmet['id'], 3)
                for product in similar:
                    print(f"   → {product['product_name']} ({product['relationship_type']}) - €{product['price']}")
            else:
                print("   Nessun helmet trovato")
        
        print("\\n2. Documenti e le loro annotazioni:")
        with kg_builder.driver.session() as session:
            docs_result = session.run("""
            MATCH (d:Document)<-[:ANNOTATION]-(a:Annotation)
            RETURN d.document_name as doc_name, 
                   collect(a.annotation_type) as annotation_types,
                   count(a) as annotation_count
            """)
            
            for record in docs_result:
                print(f"   📄 {record['doc_name']}: {record['annotation_count']} annotazioni ({', '.join(record['annotation_types'])})")
        
        print("\\n3. Prodotti per categoria:")
        with kg_builder.driver.session() as session:
            category_result = session.run("""
            MATCH (p:Product)
            WHERE p.category_name IS NOT NULL
            RETURN p.category_name as category, count(p) as product_count
            ORDER BY product_count DESC
            LIMIT 5
            """)
            
            for record in category_result:
                print(f"   🏷️ {record['category']}: {record['product_count']} prodotti")
        
        print("\\n4. Relazioni prodotto-documento:")
        with kg_builder.driver.session() as session:
            rel_result = session.run("""
            MATCH (p:Product)-[:DESCRIBED_BY]->(d:Document)
            RETURN p.name as product_name, d.document_name as document_name
            LIMIT 5
            """)
            
            for record in rel_result:
                print(f"   📦 {record['product_name']} ← → 📄 {record['document_name']}")
        
        kg_builder.close()
        
        print("\\n=== QUERY CYPHER UTILI ===")
        print("Puoi eseguire queste query nel Neo4j Browser (http://localhost:7474):")
        print()
        print("# Visualizza tutto il grafo (attenzione con grafi grandi!):")
        print("MATCH (n)-[r]-(m) RETURN n,r,m LIMIT 50")
        print()
        print("# Trova prodotti di una categoria specifica:")
        print("MATCH (p:Product) WHERE p.category_name = 'Road Bikes' RETURN p")
        print()
        print("# Trova documenti con le loro annotazioni:")
        print("MATCH (d:Document)<-[:ANNOTATION]-(a:Annotation) RETURN d,a")
        print()
        print("# Trova percorsi tra due prodotti:")
        print("MATCH path = (p1:Product)-[*1..3]-(p2:Product) WHERE p1.name CONTAINS 'Frame' AND p2.name CONTAINS 'Handlebars' RETURN path LIMIT 10")
        
    except Exception as e:
        print(f"ERRORE nelle query: {e}")
        print("Assicurati che il grafo sia stato costruito e che Neo4j sia in esecuzione.")

# Esegui le query demo solo se il grafo è già stato costruito
try:
    # Test rapido di connessione
    test_kg = KnowledgeGraphBuilder(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
    with test_kg.driver.session() as session:
        result = session.run("MATCH (n) RETURN count(n) as node_count").single()
        if result['node_count'] > 0:
            test_kg.close()
            demo_queries()
        else:
            test_kg.close()
            print("Il grafo sembra vuoto. Costruisci prima il grafo eseguendo la cella precedente con build_graph = True")
except:
    print("Neo4j non è raggiungibile o il grafo non è stato ancora costruito.")


Neo4j non è raggiungibile o il grafo non è stato ancora costruito.
